In [1]:
import numpy as np

import cv2
import time

import imutils

from directkeys import  W, A, S, D, Left, Right, Up, Down
from directkeys import PressKey, ReleaseKey 

from imutils.video import FileVideoStream, VideoStream

In [2]:
key_up = W
key_down = S
key_left = A
key_right = D


size_height = 600#360
size_width = 600#480
tracker_type = 'CSRT'

KEYS = {17:'W', 30:'A', 31:'S', 32:'D'}

In [3]:
def draw(ret, bbox, frame):
         # Draw bounding box
    if ret:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        # Tracking failure
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)


def get_centroid(bbox): #center point of rectangle
    x, y, w, h = bbox
    centx = int(x + w // 2)
    centy = int(y + h // 2)
    return (centx, centy)

def drawbox(ret, bbox, frame): #draws rectangle from bbox
    global q
    if ret:
        #Tracking Success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)

    if p1[0] < 10: #quits program if taken hand to left top corner
        q = True # quit

In [5]:
key_steer, key_acc = None, None #key to accelerate
pressed, pressed_steer = None, None

# delay = 0.3
last = 0 #last extreme of steer

COLOR_RED = (0, 0, 255)
COLOR_BLACK = (0, 0, 0)

THRESH_ACTION = 40 # threshold for a action to be registered, inside this no movement

def steer(bbox): #to steer left and right
    global last, pressed_steer, key_steer
    x, _ = get_centroid(bbox)
    diff = x - cent[0] 
    
    if abs(diff) < last: #to handle sensitivity
        if abs(diff) < last-10:
            last = 0
        return
    last = abs(diff) + 5
    
    if abs(diff) < THRESH_ACTION:
            return           

    if diff > 0:
        key_steer = key_right
    else:
        key_steer = key_left
        
    pressed_steer = True
    PressKey(key_steer) 
    
def accelerate(bbox):
    global pressed_acc, key_acc
    
    _, y = get_centroid(bbox)
    diff = - (y - cent[1])  # '-' as y decr upward

    if abs(diff) < THRESH_ACTION:         
        return
    
    if diff > 0:
        key_acc = key_up
    else:
        key_acc = key_down
        
    pressed_acc = True    
    PressKey(key_acc)
    

def draw_circle(frame, center, radius=THRESH_ACTION, color=COLOR_BLACK, thickness=3):
    cv2.circle(frame, center, radius, color, thickness)
     
def action(bbox):
    global pressed_acc, pressed_steer, key_steer, key_acc
    
    pressed_acc = False
    pressed_steer = False
    
    diff_acc = accelerate(bbox) #a
    diff_steer = steer(bbox) #s
        
    if not pressed_acc and key_acc is not None:
        ReleaseKey(key_acc)
        key_acc = None
        
    if not pressed_steer and key_steer is not None:
        ReleaseKey(key_steer)
        key_steer = None
    
    return diff_acc, diff_steer

# cv2.destroyAllWindows()

In [6]:
def get_frame():
    frame = fvs.read()
    if frame is None:
        return
    frame = cv2.flip(frame, 1)
    frame = imutils.resize(frame, width=size_width, height=size_height) 
    return frame

In [10]:
# run this only once, only to make bounding box across your hand.

fvs = FileVideoStream(0).start()
time.sleep(2.0) #to allow web cam to open

TIMER_SETUP = 3.0 # timer for capturing base image, get reading in posture
t = time.time()

while True:
    frame = get_frame()
    curr = (time.time() - t)
    if curr > TIMER_SETUP: # break when timers up
        break
    cv2.putText(frame, str(int(TIMER_SETUP - curr)+1), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4)
    cv2.imshow("Setup", frame)
    cv2.waitKey(1)

FRAME = frame.copy() 
cv2.destroyAllWindows()

# Make box around hand
cv2.putText(frame, 'Make box around Hand', (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_RED, 2)
bbox = cv2.selectROI(frame, False) # bounding box for left hand 

cent = get_centroid(bbox)
draw_circle(frame, cent) # circle, outside this movements will happend
BB = bbox # saving it for later

cv2.destroyAllWindows()
fvs.stop()

In [11]:
#re-run this each time after above cell

fvs = FileVideoStream(0).start()
bbox = BB

# Creating the CSRT Tracker
tracker = cv2.TrackerCSRT_create() # left hand for steering

# Initialize tracker with first frame and bounding box
tracker.init(FRAME, bbox)

cv2.putText(frame.copy(), 'Put both your hands in Postion', (100,70), \
    cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2)

TIMER_SETUP = 5
t = time.time()

while True:
    frame = get_frame()
    curr = (time.time() - t)
    if curr > TIMER_SETUP or frame is None:
        break
    cv2.putText(frame, str(int(TIMER_SETUP - curr)+1), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4)
    drawbox(True, bbox, frame)
    cv2.imshow("Tracking", frame)
    cv2.waitKey(1)

cv2.destroyAllWindows()


q = False #to quit, take your hand on top corner

while True:
    frame = get_frame()
    
    if frame is None:
        break
        
    ret, bbox = tracker.update(frame) # retleft - True, if found hand
    
    if ret == False:
        print("Tracking stopped")
        break
    
    action(bbox)
    
    activekeys = ' '.join([KEYS[i] for i in [key_acc, key_steer] if i])
    cv2.putText(frame, "Keys -> " + str(activekeys), (100,90), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2)
     
    drawbox(ret, bbox, frame)
    draw_circle(frame, cent)

    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK,2);
    cv2.imshow("Tracking", frame)
    
    k = cv2.waitKey(1)

    if k == 13 or q: #13 is the Enter Key
        break
        
ReleaseKey(key_steer)
ReleaseKey(key_acc)

cv2.destroyAllWindows()
fvs.stop()